In [50]:
#https://github.com/DEAP/deap/blob/82f774d9be6bad4b9d88272ba70ed6f1fca39fcf/examples/ga/knapsack.py
#    This file is part of DEAP.
#
#    DEAP is free software: you can redistribute it and/or modify
#    it under the terms of the GNU Lesser General Public License as
#    published by the Free Software Foundation, either version 3 of
#    the License, or (at your option) any later version.
#
#    DEAP is distributed in the hope that it will be useful,
#    but WITHOUT ANY WARRANTY; without even the implied warranty of
#    MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the
#    GNU Lesser General Public License for more details.
#
#    You should have received a copy of the GNU Lesser General Public
#    License along with DEAP. If not, see <http://www.gnu.org/licenses/>.

import random

import numpy

from deap import algorithms
from deap import base
from deap import creator
from deap import tools

IND_INIT_SIZE = 5
MAX_ITEM = 50
MAX_WEIGHT = 25
NBR_ITEMS = 11

# To assure reproductibility, the RNG seed is set prior to the items
# dict initialization. It is also seeded in main().
random.seed(64)

# Create the item dictionary: item name is an integer, and value is 
# a (weight, value) 2-uple.
items = {}
i_value = [100, 30, 20, 40, 50, 70, 10, 25, 30,28,30]
i_height = [7,    7,  6,   2,   5,  6,   1,   3,  10, 3, 15]
# Create random items and store them in the items' dictionary.
#for i in range(NBR_ITEMS):
for i in range(11):
    items[i] = (i_value[i], i_height[i])
    #items[i] = (random.randint(1, 10), random.uniform(0, 100))

creator.create("Fitness", base.Fitness, weights=(-1.0, 1.0))
creator.create("Individual", set, fitness=creator.Fitness)

toolbox = base.Toolbox()

# Attribute generator
toolbox.register("attr_item", random.randrange, NBR_ITEMS)

# Structure initializers
toolbox.register("individual", tools.initRepeat, creator.Individual, 
    toolbox.attr_item, IND_INIT_SIZE)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

def evalKnapsack(individual):
    weight = 0.0
    value = 0.0
    for item in individual:
        weight += items[item][0]
        value += items[item][1]
    if len(individual) > MAX_ITEM or weight > MAX_WEIGHT:
        return 10000, 0             # Ensure overweighted bags are dominated
    return weight, value

def cxSet(ind1, ind2):
    """Apply a crossover operation on input sets. The first child is the
    intersection of the two sets, the second child is the difference of the
    two sets.
    """
    temp = set(ind1)                # Used in order to keep type
    ind1 &= ind2                    # Intersection (inplace)
    ind2 ^= temp                    # Symmetric Difference (inplace)
    return ind1, ind2
    
def mutSet(individual):
    """Mutation that pops or add an element."""
    if random.random() < 0.5:
        if len(individual) > 0:     # We cannot pop from an empty set
            individual.remove(random.choice(sorted(tuple(individual))))
    else:
        individual.add(random.randrange(NBR_ITEMS))
    return individual,

toolbox.register("evaluate", evalKnapsack)
toolbox.register("mate", cxSet)
toolbox.register("mutate", mutSet)
toolbox.register("select", tools.selNSGA2)

def main():
    random.seed(64)
    NGEN = 50
    MU = 50
    LAMBDA = 100
    CXPB = 0.7
    MUTPB = 0.2
    
    pop = toolbox.population(n=MU)
    hof = tools.ParetoFront()
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", numpy.mean, axis=0)
    stats.register("std", numpy.std, axis=0)
    stats.register("min", numpy.min, axis=0)
    stats.register("max", numpy.max, axis=0)
    
    alg = algorithms.eaMuPlusLambda(pop, toolbox, MU, LAMBDA, CXPB, MUTPB, NGEN, stats,
                              halloffame=hof)
    
    return pop, stats, hof, alg, stats
      

C:\Users\matt\Anaconda3\lib\site-packages\deap\creator.py:141: RuntimeWarning: A class named 'Fitness' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
C:\Users\matt\Anaconda3\lib\site-packages\deap\creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


In [51]:
a,b,c,alg,stats = main()

gen	nevals	avg            	std    	min            	max            
0  	50    	[10000.     0.]	[0. 0.]	[10000.     0.]	[10000.     0.]
1  	87    	[9.001e+03 1.200e-01]	[2.99700250e+03 5.87877538e-01]	[0. 0.]        	[1.e+04 3.e+00]
2  	90    	[4.4033e+03 4.6000e-01]	[4.96094987e+03 1.20349491e+00]	[0. 0.]        	[1.e+04 6.e+00]
3  	91    	[1.2  0.28]            	[4.30813185 1.18389189]        	[0. 0.]        	[20.  6.]      
4  	92    	[1.   0.26]            	[4.12310563 1.18      ]        	[0. 0.]        	[20.  6.]      
5  	87    	[1.   0.26]            	[4.12310563 1.18      ]        	[0. 0.]        	[20.  6.]      
6  	83    	[1.2  0.28]            	[4.30813185 1.18389189]        	[0. 0.]        	[20.  6.]      
7  	92    	[1.2  0.28]            	[4.30813185 1.18389189]        	[0. 0.]        	[20.  6.]      
8  	88    	[1.4  0.38]            	[4.90306027 1.42674455]        	[0. 0.]        	[20.  6.]      
9  	87    	[1.4  0.38]            	[4.90306027 1.42674455]        	[0. 0.]  

In [54]:
alg[1][50]
print(len(alg[0]))
alg[0]

50


[{2},
 Individual(),
 {6},
 Individual(),
 {2},
 {2},
 {2},
 {2},
 {2},
 {2},
 {2},
 {2},
 {2},
 {2},
 {2},
 {2},
 {2},
 {2},
 {2},
 {2},
 {2},
 {2},
 {2},
 {2},
 {2},
 {2},
 {2},
 {2},
 {2},
 {2},
 {2},
 {2},
 {2},
 {2},
 {2},
 {2},
 {2},
 {2},
 {2},
 {2},
 {2},
 {2},
 {2},
 {2},
 {2},
 {2},
 {2},
 {2},
 {2},
 {2}]

In [3]:
print(a)
random.randint(1, 10)

[Individual(), Individual(), Individual(), Individual(), Individual(), Individual(), Individual(), Individual(), Individual(), Individual(), Individual(), Individual(), Individual(), Individual(), Individual(), Individual(), Individual(), Individual(), Individual(), Individual(), Individual(), Individual(), Individual(), Individual(), Individual(), Individual(), Individual(), Individual(), Individual(), Individual(), Individual(), Individual(), Individual(), Individual(), Individual(), Individual(), Individual(), Individual(), Individual(), Individual(), Individual(), Individual(), Individual(), Individual(), Individual(), Individual(), Individual(), Individual(), Individual(), Individual()]


5

In [4]:
aitems = {}
for i in range(12):
    aitems[i] = (random.randint(1, 10), random.uniform(0, 100))
aitems
random.randint(1, 10)
random.uniform(0, 100)

78.3545143760741

In [18]:
atems = {}
i_value = [100, 300, 200, 40, 500, 70, 100, 250, 300,280,300]
i_height = [7,    7,  6,   2,   5,  6,   1,   3,  10, 3, 15]
for i in range(11):
    atems[i] = (i_value[i], i_height[i])
atems

{0: (100, 7),
 1: (300, 7),
 2: (200, 6),
 3: (40, 2),
 4: (500, 5),
 5: (70, 6),
 6: (100, 1),
 7: (250, 3),
 8: (300, 10),
 9: (280, 3),
 10: (300, 15)}

In [19]:
atems[0]

(100, 7)

In [40]:

#    This file is part of DEAP.
#
#    DEAP is free software: you can redistribute it and/or modify
#    it under the terms of the GNU Lesser General Public License as
#    published by the Free Software Foundation, either version 3 of
#    the License, or (at your option) any later version.
#
#    DEAP is distributed in the hope that it will be useful,
#    but WITHOUT ANY WARRANTY; without even the implied warranty of
#    MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the
#    GNU Lesser General Public License for more details.
#
#    You should have received a copy of the GNU Lesser General Public
#    License along with DEAP. If not, see <http://www.gnu.org/licenses/>.

import random

import numpy

from deap import algorithms
from deap import base
from deap import creator
from deap import tools

IND_INIT_SIZE = 5
MAX_ITEM = 50
MAX_WEIGHT = 50
NBR_ITEMS = 20

# To assure reproductibility, the RNG seed is set prior to the items
# dict initialization. It is also seeded in main().
random.seed(64)

# Create the item dictionary: item name is an integer, and value is 
# a (weight, value) 2-uple.
items = {}
# Create random items and store them in the items' dictionary.
for i in range(NBR_ITEMS):
    items[i] = (random.randint(1, 10), random.uniform(0, 100))

creator.create("Fitness", base.Fitness, weights=(-1.0, 1.0))
creator.create("Individual", set, fitness=creator.Fitness)

toolbox = base.Toolbox()

# Attribute generator
toolbox.register("attr_item", random.randrange, NBR_ITEMS)

# Structure initializers
toolbox.register("individual", tools.initRepeat, creator.Individual, 
    toolbox.attr_item, IND_INIT_SIZE)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

def evalKnapsack(individual):
    weight = 0.0
    value = 0.0
    for item in individual:
        weight += items[item][0]
        value += items[item][1]
    if len(individual) > MAX_ITEM or weight > MAX_WEIGHT:
        return 10000, 0             # Ensure overweighted bags are dominated
    return weight, value

def cxSet(ind1, ind2):
    """Apply a crossover operation on input sets. The first child is the
    intersection of the two sets, the second child is the difference of the
    two sets.
    """
    temp = set(ind1)                # Used in order to keep type
    ind1 &= ind2                    # Intersection (inplace)
    ind2 ^= temp                    # Symmetric Difference (inplace)
    return ind1, ind2
    
def mutSet(individual):
    """Mutation that pops or add an element."""
    if random.random() < 0.5:
        if len(individual) > 0:     # We cannot pop from an empty set
            individual.remove(random.choice(sorted(tuple(individual))))
    else:
        individual.add(random.randrange(NBR_ITEMS))
    return individual,

toolbox.register("evaluate", evalKnapsack)
toolbox.register("mate", cxSet)
toolbox.register("mutate", mutSet)
toolbox.register("select", tools.selNSGA2)

def main():
    random.seed(64)
    NGEN = 50
    MU = 50
    LAMBDA = 100
    CXPB = 0.7
    MUTPB = 0.2
    
    pop = toolbox.population(n=MU)
    hof = tools.ParetoFront()
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", numpy.mean, axis=0)
    stats.register("std", numpy.std, axis=0)
    stats.register("min", numpy.min, axis=0)
    stats.register("max", numpy.max, axis=0)
    
    algorithms.eaMuPlusLambda(pop, toolbox, MU, LAMBDA, CXPB, MUTPB, NGEN, stats,
                              halloffame=hof)
    
    return pop, stats, hof

In [41]:
main()

gen	nevals	avg                        	std                      	min                      	max                        
0  	50    	[ 22.78       210.00877715]	[ 5.88316241 71.09309649]	[10.         49.69958685]	[ 38.         345.35491309]
1  	87    	[  9.96       145.20790666]	[ 10.61312395 139.1868852 ]	[0. 0.]                  	[ 45.         414.48478381]
2  	91    	[ 3.26       61.20087478]  	[  7.44797959 125.53892809]	[0. 0.]                  	[ 28.         414.48478381]
3  	88    	[ 4.6        84.51641114]  	[  8.57438044 140.51459866]	[0. 0.]                  	[ 28.         414.48478381]
4  	92    	[ 2.4        52.24310488]  	[  5.82065288 108.88598622]	[0. 0.]                  	[ 28.         414.48478381]
5  	87    	[ 3.66       74.97342258]  	[  6.99316809 127.02866009]	[0. 0.]                  	[ 28.         414.48478381]
6  	82    	[  5.3        111.43072783]	[  7.61117599 142.76899122]	[0. 0.]                  	[ 28.         414.48478381]
7  	90    	[ 3.38       76.37304048]

([{3, 4, 7, 8, 9, 10, 12, 13, 14, 15, 16, 18},
  {3, 4, 7, 8, 9, 10, 12, 13, 14, 15, 16, 18},
  Individual(),
  {2, 4, 7, 8, 10, 13, 15},
  {2, 4, 8, 10, 13, 14, 15, 18},
  {4, 7, 8, 9, 10, 12, 13, 14, 15, 16, 18},
  {4, 7, 8, 9, 10, 12, 13, 14, 15, 16, 18},
  {4, 7, 8, 9, 10, 12, 13, 14, 15, 16, 18},
  {4, 7, 8, 9, 10, 12, 13, 14, 15, 16, 18},
  {4, 7, 8, 9, 10, 12, 13, 14, 15, 16, 18},
  {4, 7, 8, 10, 12, 13, 14, 15, 16, 18},
  {3, 4, 6, 8, 9, 10, 13, 14, 15},
  {4, 7, 8, 9, 10, 12, 13, 14, 15, 18},
  Individual(),
  {10},
  {10},
  {2, 8, 10, 13},
  {4, 8, 10, 13},
  {8, 10},
  {8, 10, 13},
  {2, 4, 7, 8, 10, 13},
  {2, 4, 7, 8, 10, 13},
  {2, 4, 7, 8, 10, 13},
  {2, 4, 7, 8, 10, 13},
  {4, 8, 10, 13, 15},
  {4, 8, 10, 13, 14, 15, 18},
  {4, 7, 8, 9, 10, 13, 14, 15},
  {4, 7, 8, 9, 10, 13, 14, 15},
  {4, 7, 8, 9, 10, 13, 14, 15},
  {2, 4, 8, 10, 13, 15},
  {4, 7, 8, 9, 10, 12, 13, 14, 15, 18},
  {2, 8, 10, 13},
  {8, 10, 13},
  {4, 8, 10, 13},
  {4, 8, 10, 13, 15},
  {4, 8, 10, 13, 